In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='http://www.revistas.unam.mx/index.php/rom/issue/archive'
url_root= 'http://www.revistas.unam.mx/index.php/rom/issue/archive'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
box = soup.find('div', id='issues')
volumen=box.findAll('h4')
vol = [x.find('a').get('href')for x in volumen]
#vol=[urljoin(url_root,i) for i in vol]
vol

In [ ]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.findAll('div', id='issueCoverImage') 
    #volumen1=box1.findAll('h5',class_='summary_title_wrapper')
    vol1 = [x.find('a').get('href')for x in box1]
    #vol1=[urljoin(url_root,p) for p in vol1]
    vol2+=vol1

In [ ]:
vol2

In [ ]:
vol3 =[]
for b in vol2: 
    url_inicial2=b 
    r2 = requests.get(url_inicial2)
    soup2 = BeautifulSoup(r2.text, 'html.parser')
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('a')
    vol2= [o.get('href')for o in volumen2]
    #links = [o.find('a')['href'] for o in box2.findAll('div',class_='tocTitle')]
    #vol1=[urljoin(url_root,p) for p in vol1]
    vol3+=vol2

In [ ]:
vol3

In [ ]:
def get_url_items(sopa, url): 
    box2 = soup2.find('div', id='content') 
    volumen2=box2.findAll('div',class_='tocTitle')
    vol2 = [o.find('a').get('href')for o in volumen2]
    #vol=[urljoin(url_root,i) for i in vol]
    return vol3

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Revista Odontológica Mexicana'
    a='III. Medicina y Ciencias de la Salud'
    te='Odontología'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('h3').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', id='articleAbstract').get_text(strip=True)
        content_book['Resumen']=des.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('div', class_='panel-boody').get_text(strip=True)
        content_book['Abstract']=de#.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    try:
        urlA=url
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', id='articleFullText').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:924]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df=df_catalogo[~ df_catalogo["Titulo Articulo"].isin(["Presentación","Editorial","Autores","Introducción","Corrigenda","Comentario","Revista completa","Preliminares","Portada e índice","Enlaces refback","Mañana o pasado","Pura imagen","Número completo", None])]
df

In [ ]:
df.to_csv('Revista150.csv', index=False)